In [67]:
import requests
import json
import os
import enum
import datetime
class AggType(enum.Enum):
    day = 'day'
    week = 'week'
    month = 'month'
    quarter = 'quarter'
    year = 'year'
    hour = 'hour'
    minute = 'minute'

ticker = 'AAPL'
type =  AggType.hour.value
multiplier = 1
from_date = datetime.date(2024, 1, 1)
to_date = datetime.date(2024, 1, 2)
url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{type}/{from_date}/{to_date}'
params = {
        "apiKey": os.environ['POLYGON_API_KEY'],
        "adjusted": "true",
        "sort": "asc"
}
r = requests.get(url, params=params)
data = r.json()

print(json.dumps(data, indent=2))


KeyError: 'POLYGON_API_KEY'

In [6]:
r.request.url

'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-01-09/2023-02-10?api_key=J9VFvuEFRQ0CoJ06AHDP4zoXEbsyHVVX&adjusted=true&sort=asc'

In [15]:
len(data['Time Series (5min)'])

100

In [10]:
import os
os.environ['ALPHA_VANTAGE_API_KEY'] = "FGXR05DGQZBZ2IIT"
os.environ['ALPHA_VANTAGE_API_KEY']

'FGXR05DGQZBZ2IIT'

In [2]:
os.environ['POLYGON_API_KEY'] = "J9VFvuEFRQ0CoJ06AHDP4zoXEbsyHVVX"

In [60]:
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS, ASYNCHRONOUS
org_name = "MyOrg"
stock_bucket = "stocks"
with InfluxDBClient(url="http://127.0.0.1:8086", token='MyInitialAdminToken0==',ssl=False, verify_ssl=False) as client:
    # read_api = client.query_api()
    # tables = read_api.query(f"""
    #                         from(bucket:"{stock_bucket}")
    #                         |> range(start: -15m)
    #                         """, org=org_name)
    org_api = client.organizations_api()
    orgs = org_api.find_organizations()
    org_id = ([x for x in orgs if x.name==org_name][0].id)

In [66]:

with InfluxDBClient(url="http://127.0.0.1:8086", token='MyInitialAdminToken0==',ssl=False, verify_ssl=False) as client:
    bucket_api = client.buckets_api()
    data = {
    "org_id": org_id,
    "bucket_name": stock_bucket,
    "retention_rules": [
      {
        "type": "never",
      }
    ]
  }
    bucket_data = bucket_api.find_bucket_by_name(bucket_name=stock_bucket)
    bucket_api.delete_bucket(bucket_data.id)
    bucket_api.create_bucket( **data)

ApiException: (422)
Reason: Unprocessable Entity
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.10', 'X-Platform-Error-Code': 'conflict', 'Date': 'Thu, 03 Oct 2024 01:42:49 GMT', 'Content-Length': '77'})
HTTP response body: {
	"code": "conflict",
	"message": "bucket with name stocks already exists"
}


In [22]:
import datetime
with InfluxDBClient(url="http://127.0.0.1:8086", token='MyInitialAdminToken0==',ssl=False, verify_ssl=False) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    write_api.write(
        bucket=stock_bucket,
        org=org_name,
        data=
        [
            {
                "measurement": "stock_bar",
                "tags": {"ticker": "AAPL"},
                "fields": {"h": 1, "l":0, "c": 0.5},
                "time": int(datetime.datetime.now().timestamp()),
            }
        ],
    )
    write_api.write()

In [43]:
# get bucket
with InfluxDBClient(url="http://127.0.0.1:8086", token='MyInitialAdminToken0==',ssl=False, verify_ssl=False) as client:
    bucket_api = client.buckets_api()
    bucket = bucket_api.find_bucket_by_name(stock_bucket)
    print(bucket.name, bucket.id)

stocks bbc6165be0779323


In [91]:
import psycopg
import psycopg.sql as sql
user = "postgres"
password = "postgres"
db = "postgres"
hostname = "localhost"
port = 5432
url = f"postgresql://{user}:{password}@{hostname}:{port}"
with psycopg.connect(url, autocommit=True) as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql.SQL("CREATE DATABASE testing;"))


In [71]:
connection.close()

In [79]:
psycopg2.__version__


'2.9.9 (dt dec pq3 ext lo64)'

In [58]:
from influxdb_client import InfluxDBClient
from influxdb_client.client.write.point import Point
org = "MyOrg"
bucket = "stocks"
token = "MyInitialAdminToken0=="
query = 'from(bucket: "stocks")\
|> range(start: -10m)\
'

#establish a connection
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)

#instantiate the WriteAPI and QueryAPI
write_api = client.write_api()
query_api = client.query_api()

#create and write the point
p = Point.from_dict({
            "measurement": "b",
            "tags": {
                "test": "tag"
            },
            "fields": {
                "open": 1,
                "close": 2,
                "high": 3,
                "low": 1,
                "volume": 0
            }
        }
)
# p = Point("h2o_level").tag("location", "coyote_creek").field("water_level2", 1)
write_api.write(bucket=bucket,org=org,record=[p,p])
#return the table and print the result
result = client.query_api().query_data_frame(org=org, query=query)
results = []
print(result)

[     result  table                           _start  \
0   _result      0 2024-10-03 01:21:02.720950+00:00   
1   _result      0 2024-10-03 01:21:02.720950+00:00   
2   _result      0 2024-10-03 01:21:02.720950+00:00   
3   _result      0 2024-10-03 01:21:02.720950+00:00   
4   _result      0 2024-10-03 01:21:02.720950+00:00   
5   _result      1 2024-10-03 01:21:02.720950+00:00   
6   _result      1 2024-10-03 01:21:02.720950+00:00   
7   _result      1 2024-10-03 01:21:02.720950+00:00   
8   _result      1 2024-10-03 01:21:02.720950+00:00   
9   _result      1 2024-10-03 01:21:02.720950+00:00   
10  _result      2 2024-10-03 01:21:02.720950+00:00   
11  _result      2 2024-10-03 01:21:02.720950+00:00   
12  _result      2 2024-10-03 01:21:02.720950+00:00   
13  _result      2 2024-10-03 01:21:02.720950+00:00   
14  _result      2 2024-10-03 01:21:02.720950+00:00   
15  _result      3 2024-10-03 01:21:02.720950+00:00   
16  _result      3 2024-10-03 01:21:02.720950+00:00   
17  _resu

/home/thalsin/MarketDataML/.venv/lib/python3.12/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "stocks")|> range(start: -10m) |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


In [59]:
result[0]

,result,table,_start,_stop,_time,_value,_field,_measurement,test
0,_result,0,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:28.915272+00:00,2,close,a,tag
1,_result,0,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:31.159900+00:00,2,close,a,tag
2,_result,0,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:32.847093+00:00,2,close,a,tag
3,_result,0,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:34.266885+00:00,2,close,a,tag
4,_result,0,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:52.553234+00:00,2,close,a,tag
5,_result,1,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:28.915272+00:00,3,high,a,tag
6,_result,1,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:31.159900+00:00,3,high,a,tag
7,_result,1,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:32.847093+00:00,3,high,a,tag
8,_result,1,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:34.266885+00:00,3,high,a,tag
9,_result,1,2024-10-03 01:21:02.720950+00:00,2024-10-03 01:31:02.720950+00:00,2024-10-03 01:28:52.553234+00:00,3,high,a,tag
